In [1]:
import pickle
import pandas as pd
import numpy as np
data_dir='../data/'

In [2]:
def pathway_dict(data_dir,file='pathway-links.dat'):
    pathway_names={}
    sf=open(data_dir+file)
    for i in sf.readlines():
        lines=i.strip().split()
        pathway_names[lines[0]]=' '.join(lines[1:])
    return pathway_names

def rxn_dict(data_dir, file='reaction-links.dat'):
    rxn_ec={};rxn_no_ec=[]
    sf=open(data_dir+file)
    for i in sf.readlines():
        lines=i.strip().split()
        if len(lines)>1:
            rxn_ec[lines[0]]=(' '.join(lines[1:])).lower().replace('-',':')
        else:
            rxn_ec[lines[0]]=''
            rxn_no_ec.append(lines[0])
    return rxn_ec,rxn_no_ec

def generate_all_lines(data_dir,file='pathway_cropped.dat'):
    all_lines=[]
    sf=open(data_dir+file, encoding='utf-8',
                 errors='ignore')
    for i,j in enumerate(sf.readlines()):
        lines=j.strip()
        all_lines.append(lines)        
    return all_lines

def get_pwy_rxns(all_lines):
    pathway_rxns={}
    for i,j in enumerate(all_lines):
        if 'UNIQUE-ID' in j:
            pathway_rxns[j.split()[-1]]=[]
            for m,n in enumerate(all_lines[i+1:]):
                if 'UNIQUE-ID' not in n:
                    cleaned_line=n.replace("(",'').replace(')','').replace("\"",'')
                    pathway_rxns[j.split()[-1]]=pathway_rxns[j.split()[-1]]+[i for i in cleaned_line.split() if 'RXN' in i]
                if (i+1+m+1)<len(all_lines) and 'UNIQUE-ID' in all_lines[i+1+m+1]:
                    break
    return pathway_rxns

def get_pwy_rxnsandpwy(all_lines):
    pathway_rxnsandpwy={}
    for i,j in enumerate(all_lines):
        if 'UNIQUE-ID' in j:
            pathway_rxnsandpwy[j.split()[-1]]=[]
            for m,n in enumerate(all_lines[i+1:]):
                if 'UNIQUE-ID' not in n:
                    cleaned_line=n.replace("(",'').replace(')','').replace("\"",'')
                    pathway_rxnsandpwy[j.split()[-1]]=pathway_rxns[j.split()[-1]]+[i for i in cleaned_line.split() if ('RXN' in i) or ('PWY' in i)]
                if (i+1+m+1)<len(all_lines) and 'UNIQUE-ID' in all_lines[i+1+m+1]:
                    break
    return pathway_rxnsandpwy

def get_ec_for_pwy(pathway_rxns, rxn_ec):
    pathway_ec={}
    for i,j in pathway_rxns.items():
        pathway_ec[i]=[]
        for m,n in enumerate(j):
            if len(rxn_ec[n])>1:
                pathway_ec[i].append(rxn_ec[n])     
    return pathway_ec

def get_ec_for_pwy_all(pathway_rxnsandpwy,rxn_ec,pathway_ec):
    pathway_ec_all={}
    for i,j in pathway_rxnsandpwy.items():
        pathway_ec_all[i]=[]
        for m,n in enumerate(j):    
            if 'RXN' in n and len(rxn_ec[n])>1:
                pathway_ec_all[i].append(rxn_ec[n])  
            if 'PWY' in n and len(pathway_ec[n])>1:
                pathway_ec_all[i]=pathway_ec_all[i]+pathway_ec[n]
    return pathway_ec_all

def get_key_from_val(dict1,val):
    key=list(dict1.keys())[list(dict1.values()).index(val)]
    return key

def get_classes_dict():
    classes = pickle.load( open(data_dir+ "metacyc_classes_all.pkl", "rb" ) )
    classes_dict={}
    for i,j in classes.items():
        for m,n in enumerate(j):
            classes_dict[n]=i
    return classes_dict

def get_classes_nos_dict():
    Pathways=['Activation-Inactivation-Interconversion',
          'Bioluminescence',
          'Biosynthesis',
          'Degradation',
          'Detoxification',
          'Energy-Metabolism',
          'Glycan-Pathways',
          'Macromolecule-Modification',
             'Metabolic-Clusters',
             'Super-Pathways']
    classes_nos={j:i for i,j in enumerate(Pathways)}
    return classes_nos

def get_pathway_class_dicts(pathway_ec_all,classes_dict,classes_nos):
    pathway_class={};pathway_class_label={}
    for i,j in pathway_ec.items():
        if i in list(classes_dict.keys()):
            print (i)
            pathway_class[i]=classes_dict[i]
            pathway_class_label[i]=classes_nos[classes_dict[i]]
    return pathway_class,pathway_class_label

def create_df(pathway_ec,pathway_names,pathway_class,pathway_class_label):
    lst=[]
    for key,val in pathway_class.items():
        if len(pathway_ec[key])>3:
            lst.append([key,pathway_names[key],pathway_ec[key],list(set(pathway_ec[key])),val,pathway_class_label[key]])
    df=pd.DataFrame(lst,columns=['Map','Name','EC','EC_set','Label Name','Label'])
    return df
    
def create_df(pathway_ec,pathway_names,pathway_class,pathway_class_label):
    lst=[]
    for key,val in pathway_class.items():
        if len(pathway_ec[key])>3:
            lst.append([key,pathway_names[key],pathway_ec[key],list(set(pathway_ec[key])),val,pathway_class_label[key]])
    df=pd.DataFrame(lst,columns=['Map','Name','EC','EC_set','Label Name','Label'])
    return df

def create_df_all_labels(pathway_ec,pathway_names,Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac):
    lst=[]
    for key,val in Act.items():
        if key in list(pathway_ec.keys()) and len(pathway_ec[key])>=3:
            lst.append([key,pathway_names[key],pathway_ec[key],list(set(pathway_ec[key])),val,
                       Bio[key],Bsy[key],Deg[key],Det[key],Ene[key],Gly[key],Mac[key]])
            
    df=pd.DataFrame(lst,columns=['Map','Name','EC','EC_set','Activation','Bioluminescence',
                                'Biosynthesis','Degradation','Detoxification','Energy','Glycan','Macromolecule'])
    return df

def EC_split(ec_list):
    EC=[]
    for i in ec_list:
        j=i.split()
        for m in j:
            EC.append(m)
    return EC

def pwy_set(pwy):
    return list(set(pwy))

def get_classes_dict_all(): 
    classes = pickle.load( open(data_dir+ "metacyc_classes_all.pkl", "rb" ) )
    Act=get_classes_dict_ml('Activation-Inactivation-Interconversion',classes)
    Bio=get_classes_dict_ml('Bioluminescence',classes)
    Bsy=get_classes_dict_ml('Biosynthesis',classes)
    Deg=get_classes_dict_ml('Degradation',classes)
    Det=get_classes_dict_ml('Detoxification',classes)
    Ene=get_classes_dict_ml('Energy-Metabolism',classes)
    Gly=get_classes_dict_ml('Glycan-Pathways',classes)
    Mac=get_classes_dict_ml('Macromolecule-Modification',classes)
    Met=get_classes_dict_ml('Metabolic-Clusters',classes)
    Sup=get_classes_dict_ml('Super-Pathways',classes)
    return Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac
    
def get_classes_dict_ml(tag,classes):
    tag_o={};
    rel_list=classes[tag]
    for i,j in enumerate(list(set(pathway_names.keys()))):
        if j in rel_list:
            tag_o[j]=1
        else:
            tag_o[j]=0
    return tag_o


In [3]:

	pathway_names=PathwayDF().pathway_dict()
	rxn_ec,rxn_no_ec=PathwayDF().rxn_dict()
	all_lines=PathwayDF().generate_all_lines()
	pathway_rxns=PathwayDF().get_pwy_rxns()
	pathway_rxnsandpwy=PathwayDF().get_pwy_rxnsandpwy()
	pathway_ec=PathwayDF().get_ec_for_pwy(pathway_rxns,rxn_ec)
	pathway_ec_all=PathwayDF().get_ec_for_pwy_all(pathway_rxnsandpwy,rxn_ec,pathway_ec)
	classes_dict=PathwayDF().get_classes_dict()
	classes_nos=PathwayDF().get_classes_nos_dict()
	pathway_class,pathway_class_label=PathwayDF().get_pathway_class_dicts(pathway_ec_all,classes_dict,classes_nos)
	Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac=PathwayDF().get_classes_dict_all()
	df_all_label=create_df_all_labels(pathway_ec,pathway_names,Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac)


NameError: name 'PathwayDF' is not defined

In [8]:
pathway_names=pathway_dict(data_dir)
rxn_ec,rxn_no_ec=rxn_dict(data_dir)
all_lines=generate_all_lines(data_dir)
pathway_rxns=get_pwy_rxns(all_lines)
pathway_ec=get_ec_for_pwy(pathway_rxns,rxn_ec)
Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac=get_classes_dict_all()
df_all_label=create_df_all_labels(pathway_ec,pathway_names,Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac)


In [9]:

pathway_rxnsandpwy=get_pwy_rxnsandpwy(all_lines)
pathway_ec_all=get_ec_for_pwy_all(pathway_rxnsandpwy,rxn_ec,pathway_ec)
df_all_label_2=create_df_all_labels(pathway_ec_all,pathway_names,Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac)

#classes_dict=get_classes_dict()
#classes_nos=get_classes_nos_dict()

#pathway_class,pathway_class_label=get_pathway_class_dicts(pathway_ec_all,classes_dict,classes_nos)
#df=create_df(pathway_ec_all,pathway_names,pathway_class,pathway_class_label)

#df_old=create_df(pathway_ec,pathway_names,pathway_class,pathway_class_label)
#Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac=get_classes_dict_all()


In [13]:
df_all_label_2.shape

(2978, 12)

In [ ]:
classes = pickle.load( open(data_dir+ "metacyc_classes_all.pkl", "rb" ) )
classes.keys()

In [ ]:
df_old.head()
set(df_old['Label Name'].to_list())

In [ ]:
Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac=get_classes_dict_all()
df_all_label=create_df_all_labels(pathway_ec,pathway_names,Act,Bio,Bsy,Deg,Det,Ene,Gly,Mac)
df_all_label['EC']=df_all_label.EC.apply(EC_split)
df_all_label['EC'].iloc[1]
df_all_label[df_all_label['Map']=='PWY-5410']['EC'].to_list()
df_all_label['EC_set']=df_all_label.EC.apply(pwy_set)
df_all_label[df_all_label['Map']=='PWY-5410']['EC_set'].to_list()


In [ ]:
pickle.dump(df_all_label, open( data_dir+"df_metacyc_multilabel.pkl", "wb" ) )